In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../data/multi_asset_returns.xlsx')
data.set_index('Date',inplace=True)

# 1) Summary Statistics

In [3]:
print('Asset mean excess return')
display(data.mean())
print('Asset excess return volatility')
display(data.std())

Asset mean excess return


SPY    0.013476
EFA    0.008321
EEM    0.008241
PSP    0.013148
QAI    0.002260
HYG    0.006805
DBC    0.000735
IYR    0.013603
IEF    0.002635
BWX    0.001748
TIP    0.002922
dtype: float64

Asset excess return volatility


SPY    0.040408
EFA    0.046877
EEM    0.056732
PSP    0.062329
QAI    0.014056
HYG    0.024916
DBC    0.051458
IYR    0.052314
IEF    0.016481
BWX    0.021536
TIP    0.012953
dtype: float64

In [4]:
sharpes = (data.mean() / data.std()) * np.sqrt(12)
print("Asset with best Sharpe is: ", sharpes.idxmax() ,sharpes.max())
print("Asset with worst Sharpe is: ", sharpes.idxmin() ,sharpes.min())

Asset with best Sharpe is:  SPY 1.155296479856146
Asset with worst Sharpe is:  DBC 0.04949358760568813


# 2) MV Frontier

In [5]:
sig = data.cov()
mu = data.mean()

In [6]:
w_tan = (1 / (np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu)) * np.linalg.inv(sig) @ mu

In [7]:
w_tan

array([ 1.45263509, -0.05524948,  0.07476791, -0.14457968, -2.28330591,
        0.79843187, -0.03697442, -0.31655109,  1.54247284, -0.28618334,
        0.25453622])

In [8]:
portfolio_returns = data @ w_tan

In [9]:
print('Tangency portfolio mean returns = ',portfolio_returns.mean())
print('Tangency portfolio return vol = ',portfolio_returns.std())
print('Tangency portfolio Sharpe ratio = ',np.sqrt(12)* portfolio_returns.mean() / portfolio_returns.std())

Tangency portfolio mean returns =  0.018080079310912243
Tangency portfolio return vol =  0.028077837248828813
Tangency portfolio Sharpe ratio =  2.230628783396139


# 3) The Allocation

In [10]:
mu_p = 0.0075

In [11]:
delta = ((np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu) / (mu @ np.linalg.inv(sig) @ mu)) * mu_p
w_star = delta * w_tan
w_star

array([ 0.60258381, -0.02291866,  0.03101531, -0.05997472, -0.94716367,
        0.33120646, -0.01533777, -0.1313121 ,  0.63985042, -0.11871491,
        0.10558702])

In [12]:
portfolio_returns = data @ w_star

print('Portfolio mean returns = ',portfolio_returns.mean())
print('Portfolio return vol = ',portfolio_returns.std())
print('Portfolio Sharpe ratio = ',np.sqrt(12)* portfolio_returns.mean() / portfolio_returns.std())

Portfolio mean returns =  0.007499999999999995
Portfolio return vol =  0.011647281836817944
Portfolio Sharpe ratio =  2.2306287833961376


# 4) Long-short positions

In [13]:
data = data[['SPY','EFA']]

In [14]:
sig = data.cov()
mu = data.mean()
w_tan = (1 / (np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu)) * np.linalg.inv(sig) @ mu

mu_p = 0.0075

delta = ((np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu) / (mu @ np.linalg.inv(sig) @ mu)) * mu_p
w_star = delta * w_tan
w_star

array([ 0.84488024, -0.46701113])

In [15]:
data['EFA'] = data['EFA'] + 0.002

In [16]:
sig = data.cov()
mu = data.mean()
w_tan = (1 / (np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu)) * np.linalg.inv(sig) @ mu

mu_p = 0.0075

delta = ((np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu) / (mu @ np.linalg.inv(sig) @ mu)) * mu_p
w_star = delta * w_tan
w_star

array([ 0.83759562, -0.36700111])

# 5) Robustness

In [17]:
data = pd.read_excel('../data/multi_asset_returns.xlsx')
data.set_index('Date',inplace=True)

In [18]:
sig = data.cov()
mu = data.mean()

sig_d = np.zeros(sig.shape)
np.fill_diagonal(sig_d,sig.to_numpy().diagonal())

In [19]:
w_tan = (1 / (np.ones((1,sig_d.shape[0])) @ np.linalg.inv(sig_d) @ mu)) * np.linalg.inv(sig_d) @ mu
w_tan

array([0.10786432, 0.04948678, 0.03346283, 0.04423098, 0.14949574,
       0.1432725 , 0.00362871, 0.06495742, 0.12677535, 0.04924664,
       0.2275787 ])

# 6) Out-of-Sample Performance

In [20]:
oos = data.loc['2019-01-01':]
data = data.loc[:'2018-12-31']

In [21]:
sig = data.cov()
mu = data.mean()
w_tan = (1 / (np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu)) * np.linalg.inv(sig) @ mu

mu_p = 0.0075

delta = ((np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu) / (mu @ np.linalg.inv(sig) @ mu)) * mu_p
w_star = delta * w_tan
w_star

array([ 0.68427099, -0.02328557, -0.01476364, -0.1223508 , -0.92567426,
        0.43883476, -0.07043195, -0.15687813,  0.46354033,  0.01278168,
        0.18051016])

In [22]:
returns_insample = data @ w_star
print("In sample Sharpe = ", np.sqrt(12) * returns_insample.mean() / returns_insample.std())

returns_oos = oos @ w_star
print("Out of sample Sharpe = ", np.sqrt(12) * returns_oos.mean() / returns_oos.std())

In sample Sharpe =  2.393472529246011
Out of sample Sharpe =  1.766822102591454


# 7) Robust Out-of-Sample Performance

In [23]:
sig = data.cov()
mu = data.mean()

sig_d = np.zeros(sig.shape)
np.fill_diagonal(sig_d,sig.to_numpy().diagonal())

w_tan = (1 / (np.ones((1,sig_d.shape[0])) @ np.linalg.inv(sig_d) @ mu)) * np.linalg.inv(sig_d) @ mu

mu_p = 0.0075

delta = ((np.ones((1,sig.shape[0])) @ np.linalg.inv(sig) @ mu) / (mu @ np.linalg.inv(sig) @ mu)) * mu_p
w_star = delta * w_tan
w_star

array([ 0.06007842,  0.02306873,  0.01431151,  0.02272127,  0.06982367,
        0.08016955, -0.00488151,  0.03430989,  0.05866074,  0.02130272,
        0.08698857])

In [24]:
returns_insample = data @ w_star
print("In sample Sharpe = ", np.sqrt(12) * returns_insample.mean() / returns_insample.std())

returns_oos = oos @ w_star
print("Out of sample Sharpe = ", np.sqrt(12) * returns_oos.mean() / returns_oos.std())

In sample Sharpe =  1.044438985268334
Out of sample Sharpe =  1.1991481815756602
